In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] =  '0' #'3,2,1,0'
import sys
sys.path.append("..")
from common import *
from data   import *
from imgaug import augmenters as iaa

##----------------------------------------
from model_seresnext50 import *
# from model_resnet34 import *

TkAgg
@common.py:  
	set random seed
		SEED=35202
	set cuda environment
		torch.__version__              = 0.4.1
		torch.version.cuda             = 9.0
		torch.backends.cudnn.version() = 7005
		os['CUDA_VISIBLE_DEVICES']     = 0
		torch.cuda.device_count()      = 1



In [2]:
FILE_NAME = 'baseline_25k'

In [3]:
def valid_augment(drawing, label, index):
#     image = drawing_to_image_with_color_v2(drawing, 96, 96)
    seq = iaa.Sequential([
    iaa.Crop(percent=(0.05, 0.05, \
                      0.05, 0.05), keep_size=True)
    ])
    image = drawing_to_image_with_color_aug(drawing, 96, 96, seq)
    return image, label, None


def train_augment(drawing, label, index):
    up_rand = np.random.random()
    right_rand = np.random.random()
    percent_crop = 0.1
    seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(up_rand*percent_crop, right_rand*percent_crop, \
                      (1-up_rand)*percent_crop, (1-right_rand)*percent_crop), keep_size=True)
    ])
    
    image = drawing_to_image_with_color_aug(drawing, 96, 96, seq)
    return image, label, None


In [4]:
### training ##############################################################

def do_valid( net, valid_loader, criterion ):

    valid_num  = 0
    probs    = []
    truths   = []
    losses   = []
    corrects = []

    for input, truth, cache in valid_loader:
        input = input.cuda()
        truth = truth.cuda()

        with torch.no_grad():
            logit   = net(input)
            prob    = F.softmax(logit,1)

            loss    = criterion(logit, truth, False)
            correct = metric(logit, truth, False)

        valid_num += len(input)
        probs.append(prob.data.cpu().numpy())
        losses.append(loss.data.cpu().numpy())
        corrects.append(correct.data.cpu().numpy())
        truths.append(truth.data.cpu().numpy())


    assert(valid_num == len(valid_loader.sampler))
    #------------------------------------------------------
    prob    = np.concatenate(probs)
    correct = np.concatenate(corrects)
    truth   = np.concatenate(truths).astype(np.int32).reshape(-1,1)
    loss    = np.concatenate(losses)


    #---
    #top = np.argsort(-predict,1)[:,:3]

    loss    = loss.mean()
    correct = correct.mean(0)

    top = [correct[0], correct[0]+correct[1], correct[0]+correct[1]+correct[2]]
    precision = correct[0]/1 + correct[1]/2 + correct[2]/3

    #----
    valid_loss = np.array([
        loss, top[0], top[2], precision
    ])

    return valid_loss

In [5]:
fold    = 0
out_dir = \
    '../../output'
initial_checkpoint = None
#         '../../output/backup/873_crop.pth'

pretrain_file = None #pre-trained model will be loaded automatically, thus do not have to set here.

batch_size = 256+64
epoch = 6
num_iters   = epoch * 340 * 25000 // batch_size

#     schduler  = NullScheduler(lr=0.01)
schduler = DecayScheduler(base_lr=0.01, decay=0.1, step=num_iters/2) #it means 3 epoch for 0.01, 3 epoch for 0.001
iter_save_interval = 2000
criterion          = softmax_cross_entropy_criterion


## setup  -----------------------------------------------------------------------------
os.makedirs(out_dir +'/checkpoint', exist_ok=True)
os.makedirs(out_dir +'/train', exist_ok=True)
os.makedirs(out_dir +'/backup', exist_ok=True)
#     backup_project_as_zip(PROJECT_PATH, out_dir +'/backup/code.train.%s.zip'%IDENTIFIER)

log = Logger()
log.open(out_dir+'/log.train_r50_add_crop.txt',mode='a')
log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
log.write('\tSEED         = %u\n' % SEED)
log.write('\tPROJECT_PATH = %s\n' % PROJECT_PATH)
log.write('\t__file__     = %s\n' % FILE_NAME)
log.write('\tout_dir      = %s\n' % out_dir)
log.write('\n')
log.write('\t<additional comments>\n')
log.write('\t  ... xxx baseline  ... \n')
log.write('\n')


## dataset ----------------------------------------
log.write('** dataset setting **\n')

train_dataset = DoodleDataset('train', 'train_0', train_augment)
train_loader  = DataLoader(
                    train_dataset,
                    #sampler     = FixLengthRandomSamplerWithProbability(train_dataset, probability),
                    #sampler     = FixLengthRandomSampler(train_dataset),
                    #sampler     = ConstantSampler(train_dataset,[31]*batch_size*100),
                    sampler     = RandomSampler(train_dataset),
                    batch_size  = batch_size,
                    drop_last   = True,
                    pin_memory  = True,
                    collate_fn  = null_collate)

valid_dataset = DoodleDataset('train', 'valid_0',  valid_augment)
valid_loader  = DataLoader(
                    valid_dataset,
                    #sampler     = SequentialSampler(valid_dataset),
                    sampler     = RandomSampler(valid_dataset),
                    batch_size  = batch_size,
                    drop_last   = False,
                    pin_memory  = True,
                    collate_fn  = null_collate)


assert(len(train_dataset)>=batch_size)
log.write('batch_size = %d\n'%(batch_size))
log.write('train_dataset : \n%s\n'%(train_dataset))
log.write('valid_dataset : \n%s\n'%(valid_dataset))
log.write('\n')

## net ----------------------------------------
log.write('** net setting **\n')
net = Net().cuda()

if initial_checkpoint is not None:
    log.write('\tinitial_checkpoint = %s\n' % initial_checkpoint)
    net.load_state_dict(torch.load(initial_checkpoint, map_location=lambda storage, loc: storage))


--- [START 2018-11-13_19-52-16] ----------------------------------------------------------------

	SEED         = 35202
	PROJECT_PATH = C:\Users\Admin\Desktop\doodle\code
	__file__     = baseline_25k
	out_dir      = ../../output

	<additional comments>
	  ... xxx baseline  ... 

** dataset setting **
	 load df   :  339/340                   zigzag   0 min 39 sec
	 load split:  339/340                   zigzag   0 min 41 sec

	 load df   :  339/340                   zigzag   0 min 00 sec
	 load split:  339/340                   zigzag   0 min 01 sec

batch_size = 320
train_dataset : 
	split        = train_0
	mode         = train
	complexity   = simplified
	len(self.id) = 8500000


valid_dataset : 
	split        = valid_0
	mode         = train
	complexity   = simplified
	len(self.id) = 27200



** net setting **
Load imagenet pretrained SERESNEXT50
	initial_checkpoint = ../../output/backup/873_crop.pth


In [6]:
log.write('%s\n'%(type(net)))
log.write('criterion=%s\n'%criterion)
log.write('\n')


## optimiser ----------------------------------
if 0: ##freeze
    for p in net.resnet.parameters(): p.requires_grad = False
    for p in net.encoder1.parameters(): p.requires_grad = False
    for p in net.encoder2.parameters(): p.requires_grad = False
    for p in net.encoder3.parameters(): p.requires_grad = False
    for p in net.encoder4.parameters(): p.requires_grad = False
    pass

#net.set_mode('train',is_freeze_bn=True)
#-----------------------------------------------


optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()),
                      lr=schduler.get_rate(0), momentum=0.9, weight_decay=0.0001)


iter_smooth = 20
iter_log    = 50
iter_valid  = 100
iter_save   = [0, num_iters-1]\
               + list(range(0, num_iters, iter_save_interval))#1*1000

start_iter = 0
start_epoch= 0
rate       = 0
if initial_checkpoint is not None:
#     initial_optimizer = initial_checkpoint.replace('_model.pth','_optimizer.pth')
#     checkpoint  = torch.load(initial_optimizer)
#     start_iter  = checkpoint['iter' ]
#     start_epoch = checkpoint['epoch']

    #rate = get_learning_rate(optimizer)  #load all except learning rate
    #optimizer.load_state_dict(checkpoint['optimizer'])
    #adjust_learning_rate(optimizer, rate)
    pass




log.write('schduler\n  %s\n'%(schduler))
log.write('\n')

## start training here! ##############################################
log.write('** start training here! **\n')
log.write('                    |------------ VALID -------------|-------- TRAIN/BATCH ----------|         \n')
log.write('rate   iter  epoch  | loss   acc-1  acc-3   lb       | loss   acc-1  acc-3   lb      |  time   \n')
log.write('----------------------------------------------------------------------------------------------------\n')


train_loss   = np.zeros(6,np.float32)
valid_loss   = np.zeros(6,np.float32)
batch_loss   = np.zeros(6,np.float32)
iter = 0
i    = 0
last_max_lb   = -1


start = timer()
while  iter<num_iters:
    sum_train_loss = np.zeros(6,np.float32)
    sum = 0


    optimizer.zero_grad()
    for input, truth, cache in train_loader:

        len_train_dataset = len(train_dataset)
        batch_size = len(cache)
        iter = i + start_iter
        epoch = (iter-start_iter)*batch_size/len_train_dataset + start_epoch
        num_samples = epoch*len_train_dataset


        if (iter % iter_valid==0) and (iter!=0):
            net.set_mode('valid')
            valid_loss = do_valid(net, valid_loader, criterion)
            net.set_mode('train')

            ##--------
            # lb    = valid_loss[7]
            # loss  = valid_loss[0] + valid_loss[4]
            # last_max_lb = max(last_max_lb,lb)
            # if last_max_lb-lb<0.005:
            #     iter_save += [iter,]
            # if loss-last_min_loss<0.005:
            #     iter_save += [iter,]

            asterisk = '*' if iter in iter_save else ' '
            ##--------

            print('\r',end='',flush=True)
            log.write('%0.4f %5.1f %6.1f | %0.3f  %0.3f  %0.3f  (%0.3f)%s  | %0.3f  %0.3f  %0.3f  (%0.3f)  | %s' % (\
                     rate, iter/1000, epoch,
                     valid_loss[0], valid_loss[1], valid_loss[2], valid_loss[3],asterisk,
                     train_loss[0], train_loss[1], train_loss[2], train_loss[3],
                     time_to_str((timer() - start),'min'))
            )
            log.write('\n')
            time.sleep(0.01)

        #if 0:
        if iter in iter_save:
            torch.save(net.state_dict(),out_dir +'/checkpoint/%08d_model.pth'%(iter))
            torch.save({
                #'optimizer': optimizer.state_dict(),
                'iter'     : iter,
                'epoch'    : epoch,
            }, out_dir +'/checkpoint/%08d_optimizer.pth'%(iter))
            pass




        # learning rate schduler -------------
        lr = schduler.get_rate(iter)
        if lr<0 : break
        adjust_learning_rate(optimizer, lr)
        rate = get_learning_rate(optimizer)



        # one iteration update  -------------
        #net.set_mode('train',is_freeze_bn=True)
        net.set_mode('train')
        input = input.cuda()
        truth = truth.cuda()

        logit = data_parallel(net, input)
        loss  = criterion(logit, truth)
        precision, top = metric(logit, truth)


        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        #torch.nn.utils.clip_grad_norm(net.parameters(), 1)


        # print statistics  ------------
        batch_loss[:4] = np.array(( loss.item(), top[0].item(), top[2].item(), precision.item(),))
        sum_train_loss += batch_loss
        sum += 1
        if iter%iter_smooth == 0:
            train_loss = sum_train_loss/sum
            sum_train_loss = np.zeros(6,np.float32)
            sum = 0


        print('\r',end='',flush=True)
        print('%0.4f %5.1f %6.1f | %0.3f  %0.3f  %0.3f  (%0.3f)%s  | %0.3f  %0.3f  %0.3f  (%0.3f)  | %s' % (\
                     rate, iter/1000, epoch,
                     valid_loss[0], valid_loss[1], valid_loss[2], valid_loss[3],' ',
                     batch_loss[0], batch_loss[1], batch_loss[2], batch_loss[3],
                     time_to_str((timer() - start),'min'))
        , end='',flush=True)
        i=i+1



    pass  #-- end of one data loader --
pass #-- end of all iterations --


if 1: #save last
    torch.save(net.state_dict(),out_dir +'/checkpoint/%d_model.pth'%(i))
    torch.save({
        'optimizer': optimizer.state_dict(),
        'iter'     : i,
        'epoch'    : epoch,
    }, out_dir +'/checkpoint/%d_optimizer.pth'%(i))

log.write('\n')



<class 'model_seresnext50.Net'>
criterion=<function softmax_cross_entropy_criterion at 0x000001AEFC86A0D0>

schduler
  (Exp) Decay Learning Rates
base_lr=0.001, decay=0.100, step=53125.000

** start training here! **
                    |------------ VALID -------------|-------- TRAIN/BATCH ----------|         
rate   iter  epoch  | loss   acc-1  acc-3   lb       | loss   acc-1  acc-3   lb      |  time   
----------------------------------------------------------------------------------------------------


D:\Anaconda\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


0.0010   0.1    0.0 | 0.000  0.000  0.000  (0.000)   | 0.747  0.831  0.925  (0.874)  |  0 hr 01 min

D:\Anaconda\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


0.0010   0.1    0.0 | 0.659  0.830  0.939  (0.881)   | 0.632  0.833  0.947  (0.886)  |  0 hr 02 min
0.0010   0.2    0.0 | 0.658  0.830  0.939  (0.880)   | 0.651  0.836  0.936  (0.882)  |  0 hr 04 min
0.0010   0.3    0.0 | 0.659  0.829  0.939  (0.880)   | 0.668  0.832  0.937  (0.880)  |  0 hr 06 min
0.0010   0.4    0.0 | 0.658  0.830  0.939  (0.881)   | 0.655  0.831  0.937  (0.880)  |  0 hr 09 min
0.0010   0.5    0.0 | 0.659  0.830  0.939  (0.881)   | 0.644  0.837  0.942  (0.886)  |  0 hr 11 min
0.0010   0.6    0.0 | 0.658  0.830  0.939  (0.881)   | 0.664  0.833  0.940  (0.882)  |  0 hr 13 min
0.0010   0.7    0.0 | 0.658  0.829  0.940  (0.881)   | 0.645  0.838  0.940  (0.885)  |  0 hr 15 min
0.0010   0.8    0.0 | 0.657  0.830  0.940  (0.881)   | 0.663  0.834  0.939  (0.883)  |  0 hr 18 min
0.0010   0.9    0.0 | 0.657  0.831  0.939  (0.881)   | 0.611  0.838  0.946  (0.888)  |  0 hr 20 min
0.0010   1.0    0.0 | 0.657  0.830  0.940  (0.881)   | 0.672  0.830  0.938  (0.880)  |  0 hr 22 min


0.0010  16.3    0.6 | 0.643  0.835  0.942  (0.885)   | 0.634  0.837  0.940  (0.885)  |  6 hr 09 min
0.0010  16.4    0.6 | 0.643  0.836  0.942  (0.885)   | 0.604  0.844  0.945  (0.891)  |  6 hr 11 min
0.0010  16.5    0.6 | 0.643  0.835  0.942  (0.885)   | 0.621  0.839  0.944  (0.888)  |  6 hr 13 min
0.0010  16.6    0.6 | 0.642  0.836  0.942  (0.885)   | 0.643  0.837  0.940  (0.885)  |  6 hr 16 min
0.0010  16.7    0.6 | 0.642  0.835  0.942  (0.885)   | 0.614  0.847  0.944  (0.892)  |  6 hr 18 min
0.0010  16.8    0.6 | 0.643  0.835  0.942  (0.885)   | 0.620  0.845  0.943  (0.890)  |  6 hr 20 min
0.0010  16.9    0.6 | 0.643  0.835  0.942  (0.884)   | 0.613  0.842  0.943  (0.888)  |  6 hr 23 min
0.0010  17.0    0.6 | 0.641  0.835  0.942  (0.885)   | 0.621  0.842  0.945  (0.891)  |  6 hr 25 min
0.0010  17.1    0.6 | 0.642  0.835  0.941  (0.884)   | 0.608  0.846  0.944  (0.892)  |  6 hr 27 min
0.0010  17.2    0.6 | 0.642  0.835  0.941  (0.884)   | 0.619  0.841  0.942  (0.888)  |  6 hr 29 min


0.0010  32.5    1.2 | 0.637  0.837  0.942  (0.886)   | 0.622  0.842  0.944  (0.889)  | 12 hr 16 min
0.0010  32.6    1.2 | 0.637  0.837  0.943  (0.886)   | 0.573  0.848  0.953  (0.896)  | 12 hr 18 min
0.0010  32.7    1.2 | 0.638  0.835  0.943  (0.885)   | 0.597  0.847  0.945  (0.892)  | 12 hr 20 min
0.0010  32.8    1.2 | 0.638  0.836  0.942  (0.885)   | 0.603  0.841  0.945  (0.890)  | 12 hr 22 min
0.0010  32.9    1.2 | 0.638  0.836  0.943  (0.886)   | 0.612  0.848  0.946  (0.893)  | 12 hr 25 min
0.0010  33.0    1.2 | 0.636  0.836  0.943  (0.885)   | 0.608  0.843  0.944  (0.890)  | 12 hr 27 min
0.0010  33.1    1.2 | 0.637  0.836  0.942  (0.885)   | 0.609  0.843  0.943  (0.890)  | 12 hr 29 min
0.0010  33.2    1.2 | 0.637  0.836  0.942  (0.885)   | 0.583  0.848  0.947  (0.894)  | 12 hr 31 min
0.0010  33.3    1.3 | 0.637  0.837  0.942  (0.886)   | 0.570  0.858  0.950  (0.901)  | 12 hr 34 min
0.0010  33.4    1.3 | 0.638  0.837  0.943  (0.886)   | 0.577  0.852  0.951  (0.898)  | 12 hr 36 min


0.0010  48.7    1.8 | 0.638  0.836  0.943  (0.885)   | 0.527  0.864  0.953  (0.905)  | 18 hr 22 min
0.0010  48.8    1.8 | 0.638  0.837  0.943  (0.886)   | 0.539  0.857  0.952  (0.902)  | 18 hr 25 min
0.0010  48.9    1.8 | 0.640  0.836  0.943  (0.886)   | 0.537  0.860  0.951  (0.902)  | 18 hr 27 min
0.0010  49.0    1.8 | 0.639  0.836  0.942  (0.885)   | 0.539  0.862  0.951  (0.903)  | 18 hr 29 min
0.0010  49.1    1.8 | 0.638  0.836  0.943  (0.886)   | 0.563  0.857  0.949  (0.900)  | 18 hr 31 min
0.0010  49.2    1.9 | 0.637  0.836  0.943  (0.885)   | 0.550  0.860  0.950  (0.902)  | 18 hr 34 min
0.0010  49.3    1.9 | 0.638  0.835  0.943  (0.885)   | 0.512  0.870  0.955  (0.909)  | 18 hr 36 min
0.0010  49.4    1.9 | 0.638  0.836  0.943  (0.886)   | 0.546  0.860  0.948  (0.902)  | 18 hr 38 min
0.0010  49.5    1.9 | 0.638  0.837  0.943  (0.886)   | 0.577  0.850  0.949  (0.896)  | 18 hr 40 min
0.0010  49.6    1.9 | 0.637  0.836  0.943  (0.886)   | 0.567  0.852  0.949  (0.897)  | 18 hr 43 min


0.0001  64.9    2.4 | 0.633  0.838  0.942  (0.887)   | 0.610  0.840  0.944  (0.888)  | 24 hr 29 min
0.0001  65.0    2.4 | 0.634  0.838  0.943  (0.887)   | 0.611  0.845  0.944  (0.891)  | 24 hr 31 min
0.0001  65.1    2.5 | 0.634  0.838  0.942  (0.886)   | 0.585  0.853  0.948  (0.897)  | 24 hr 34 min
0.0001  65.2    2.5 | 0.634  0.838  0.943  (0.887)   | 0.598  0.847  0.946  (0.893)  | 24 hr 36 min
0.0001  65.3    2.5 | 0.634  0.837  0.943  (0.886)   | 0.623  0.842  0.942  (0.889)  | 24 hr 38 min
0.0001  65.4    2.5 | 0.634  0.837  0.942  (0.886)   | 0.594  0.847  0.949  (0.894)  | 24 hr 40 min
0.0001  65.5    2.5 | 0.633  0.838  0.943  (0.886)   | 0.599  0.848  0.947  (0.894)  | 24 hr 43 min
0.0001  65.6    2.5 | 0.633  0.838  0.943  (0.887)   | 0.582  0.851  0.949  (0.897)  | 24 hr 45 min
0.0001  65.7    2.5 | 0.633  0.838  0.943  (0.886)   | 0.617  0.843  0.944  (0.889)  | 24 hr 47 min
0.0001  65.8    2.5 | 0.633  0.837  0.942  (0.886)   | 0.592  0.845  0.944  (0.892)  | 24 hr 49 min


KeyboardInterrupt: 

In [ ]:
input.shape